# Aries Basic Controller - Openmined PryVote Community (Alice)

This notebook walks through a MVP for PryVote assuming the Openminded community will be issued an Openmined commuity credential that will enable community members to vote on basic things. The credential will be issued across a previously established connection. 

It is best run in parallel with the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) controlling the agent receiving the credential to enable voting.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [16]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [17]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    

## Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [18]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)


## Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)

In [19]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'invitation_key': '7YXVNT5gB6XaUbV9k4mh2kWyCkCq1Gs6LPubfkAMWn2Q', 'my_did': 'AUBf52WokdHcqpf7G7MKL3', 'state': 'active', 'initiator': 'external', 'their_did': 'UdYwL5Po1kwJExjpvMAQ5J', 'their_label': 'Bob', 'created_at': '2020-08-17 08:36:11.734425Z', 'updated_at': '2020-08-17 08:36:21.886075Z', 'request_id': 'e41e3e7a-56f6-4c11-bcc4-03b0666fca0f', 'accept': 'manual', 'invitation_mode': 'once', 'routing_state': 'none', 'connection_id': 'cc83199a-829b-4781-94fe-54a88efe9f32'}]
Connection : {'invitation_key': '7YXVNT5gB6XaUbV9k4mh2kWyCkCq1Gs6LPubfkAMWn2Q', 'my_did': 'AUBf52WokdHcqpf7G7MKL3', 'state': 'active', 'initiator': 'external', 'their_did': 'UdYwL5Po1kwJExjpvMAQ5J', 'their_label': 'Bob', 'created_at': '2020-08-17 08:36:11.734425Z', 'updated_at': '2020-08-17 08:36:21.886075Z', 'request_id': 'e41e3e7a-56f6-4c11-bcc4-03b0666fca0f', 'accept': 'manual', 'invitation_mode': 'once', 'routing_state': 'none', 'connection_id': 'cc83199a-829b-4781-94fe-54a88efe9f32'}
Active Conne

## Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

In [20]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_community-member"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "community_member"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member:0.0.1


## Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

In [21]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default


**Note: You should be able to see both schema and definition transactions on the local network [here](http://localhost:9000)**

## Populate the Attribues to Issue to Openmind-pryvote-voter (Bob)

The schema defines two attributes: name and community_member. Both must be populated with attribute values that Openmined Community (Alice) wishes to issue in a credential to Openmind-pryvote-voter (Bob). To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [ ]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "community_member", "value": "1"}
]
print(credential_attributes)

## Send Credential

This sends a credential to a holder Openmind-pryvote-voter (Bob), and automates the rest of the protocol. This tutorial works best if you have initialised the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) and registered the listener for the issue_credential webhook.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [ ]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


## Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

Each record has a state, representing the stage of the protocol the record is at and a role. Either issuer or holder.

In [ ]:
response = await agent_controller.issuer.get_records()
print(response['results'])

In [ ]:
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

## Remove Credential Exchange

It is possible to remove stored credential exchanges from an agent. Typically when you are not interested in the credential being offered, or the credential offer that you have sent has become stale (been ignored?).

In [ ]:
response = await agent_controller.issuer.remove_record(record_id)
print(response)

In [ ]:
response = await agent_controller.issuer.get_records()
print(response)

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

There are a few api calls not covered in this tutorial so far. Mostly around credential revocation.

In [ ]:
response = await agent_controller.terminate()
print(response)

## Populate the Attribues to Issue to Openmind-pryvote-voter (Bob)

The schema defines two attributes: name and community_member. Both must be populated with attribute values that Openmined Community (Alice) wishes to issue in a credential to Openmind-pryvote-voter (Bob). To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [22]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "community_member", "value": "1"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## Send Credential

This sends a credential to a holder Openmind-pryvote-voter (Bob), and automates the rest of the protocol. This tutorial works best if you have initialised the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) and registered the listener for the issue_credential webhook.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [23]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]}, 'connection_id': 'cc83199a-829b-4781-94fe-54a88efe9f32', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member:0.0.1', 'schema_name': 'open_mined_community-member', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
Credential exchange 5c8c0b23-a936-4e9b-aa14-a078909de453, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 5c8c0b23-a936-4e9b-aa14-a078909de453, role: issuer, state: offer_sent
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

Each record has a state, representing the stage of the protocol the record is at and a role. Either issuer or holder.

In [24]:
response = await agent_controller.issuer.get_records()
print(response['results'])

[{'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default', 'key_correctness_proof': {'c': '75246854600138263239318692808331785354574439553574205542960776927894460165496', 'xz_cap': '490429143211028175570596885238678962312478505868680762296839921850619745537859838582009784553281970502457913406605806625000728158998569150440471898714339685694522187045222790844690334771841782523775439935501011551701051080361152931798268076119089451744702640081504264949465855957269660851853117876255092575623807483257136226869730066586195742477599043506719399623068456276889938216863914240152667209889734443060603742625053634803071571710192206841365860998009546756386864006493312378397112912812638539449855110045008269539802198649724192760519011360670149836170898953027967182348001020400681887540325262495349189051038148501071157291189228056931220470292724260302157029592171609006470786983442', 'xr_cap': [['name', '12571

In [25]:
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 5c8c0b23-a936-4e9b-aa14-a078909de453, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 5c8c0b23-a936-4e9b-aa14-a078909de453, role: issuer, state: request_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange 5c8c0b23-a936-4e9b-aa14-a078909de453, role: issuer, state: credential_issued
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange 5c8c0b23-a936-4e9b-aa14-a078909de453, role: issuer, state: credential_acked
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## Remove Credential Exchange

It is possible to remove stored credential exchanges from an agent. Typically when you are not interested in the credential being offered, or the credential offer that you have sent has become stale (been ignored?).

In [13]:
response = await agent_controller.issuer.remove_record(record_id)
print(response)

{}


In [29]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': []}


## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

There are a few api calls not covered in this tutorial so far. Mostly around credential revocation.

In [15]:
response = await agent_controller.terminate()
print(response)

None
